In [1]:
import pandas as pd
import numpy as np
import folium
from shapely.geometry import Point, LineString
from shapely.affinity import scale
import branca.colormap as cm

In [2]:
# 1. 데이터 로딩
df_all = pd.read_csv('C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\db\\최종사용 데이터\\jeju_with_coords_kakao.csv').dropna(
    subset=['출발_lat', '출발_lon', '도착_lat', '도착_lon'])

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\db\\최종사용 데이터\\jeju_with_coords_kakao.csv'

In [ ]:
# 2. 타원형 회피 경계 정의
center = Point(126.55, 33.368)
circle = center.buffer(0.0825, resolution=80)  # 약 9.15km
ellipse = scale(circle, xfact=2.0, yfact=0.75, origin=center)

In [ ]:
# 3. 보조 함수들

def count_intersections(line, polygon):
    inter = line.intersection(polygon)
    if inter.is_empty:
        return 0
    if inter.geom_type == 'Point':
        return 1
    elif inter.geom_type == 'MultiPoint':
        return len(inter.geoms)
    elif inter.geom_type == 'LineString':
        return 2
    elif inter.geom_type == 'MultiLineString':
        return sum(1 for g in inter.geoms if g.length > 0)
    return 3


def find_valid_random_midpoint(start, end, barrier, max_attempts=20, initial_buffer=0.005, step=0.003, samples=100):
    for i in range(max_attempts):
        dist = initial_buffer + i * step
        buffer_ring = barrier.buffer(dist).exterior
        candidate_pts = list(buffer_ring.coords)[::max(
            1, len(buffer_ring.coords) // samples)]
        for pt in candidate_pts:
            mid = Point(pt)
            l1, l2 = LineString([start, mid]), LineString([mid, end])
            if count_intersections(l1, barrier) <= 1 and count_intersections(l2, barrier) <= 1:
                return mid, dist
    return None, None


In [ ]:
# 4. 경로 생성 + 100m 간격 점 추출
records = []
sampled_points = []
route_lines = []

for _, row in df_all.iterrows():
    sp = Point(row['출발_lon'], row['출발_lat'])
    ep = Point(row['도착_lon'], row['도착_lat'])
    direct = LineString([sp, ep])

    # 경로 회피 여부 판단
    if not direct.intersects(ellipse):
        route = direct
        rerouted = False
        mid = None
    else:
        mid, dist = find_valid_random_midpoint(sp, ep, ellipse)
        if mid:
            route = LineString([sp, mid, ep])
            rerouted = True
        else:
            route = direct
            rerouted = False

    # 경로 저장
    route_lines.append(route)
    records.append({
        '출발_lat': sp.y, '출발_lon': sp.x,
        '도착_lat': ep.y, '도착_lon': ep.x,
        '우회_lat': mid.y if mid else None,
        '우회_lon': mid.x if mid else None,
        '우회_성공': rerouted,
    })

    # 경로에서 100m 간격 점 추출
    total_length = route.length
    num_points = int(total_length // 0.001)
    for i in range(num_points + 1):
        pt = route.interpolate(i / max(1, num_points), normalized=True)
        sampled_points.append((pt.y, pt.x))

In [ ]:
# 5. 결과 저장

reroute_df = pd.DataFrame(records)
sampled_df = pd.DataFrame(sampled_points, columns=['lat', 'lon'])

reroute_df.to_csv("C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\rerouted_paths.csv", index=False)
sampled_df.to_csv("C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\route_points_sampled_100m.csv", index=False)



In [ ]:
# 6. 결과 시각화 1: 경로 선 (파란 선)

m1 = folium.Map(location=[33.38, 126.55], zoom_start=11)
for route in route_lines:
    folium.PolyLine([(pt[1], pt[0]) for pt in route.coords],
                    color="blue", weight=2.5, opacity=0.7).add_to(m1)
m1.save("C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\01_rerouted_paths_all_blue.html")



In [ ]:
# 7. 결과 시각화 2: skaterbox-style 점 밀도 시각화

# 포인트 중복 수(count)를 기준으로 진하기 결정
# 100m 간격으로 추출된 포인트를 그룹화하여 count 계산

heat_counts = sampled_df.groupby(
    ['lat', 'lon']).size().reset_index(name='count')

# 지도 생성
m2 = folium.Map(location=[33.38, 126.55], zoom_start=11)
for _, row in heat_counts.iterrows():
    norm_opacity = min(
        1.0, 0.2 + 0.8 * row['count'] / heat_counts['count'].max())
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=2,
        color='blue',
        fill=True,
        fill_opacity=norm_opacity,
        weight=0
    ).add_to(m2)

m2.save("C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\02_skaterbox_point_heatmap.html")

In [ ]:
# '충전기ID' 기준으로 충전소별 충전기 개수 집계하여 시각화 전체 흐름 재수행
# 수요는 route_points_sampled_100m.csv, 공급은 '충전소명' 기준 충전기ID 개수

In [ ]:
# 수요 데이터 로드
df_demand = pd.read_csv(
    'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\route_points_sampled_100m.csv')

In [ ]:
# 공급 데이터 가공: 충전소별 충전기 수량 집계
df_chargers = pd.read_csv(
    'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\db\\최종사용 데이터\\제주도_충전소_with_coords.csv')
df_charger_count = df_chargers.groupby(['충전소명', 'lat', 'lon'])[
    '충전기ID'].nunique().reset_index(name='충전기수')
# 충전소별 충전기 수량 집계

# 충전기 수만큼 공급 좌표 확장
supply_points = np.repeat(df_charger_count[['lat', 'lon']].values,
                          df_charger_count['충전기수'].astype(int), axis=0)

In [ ]:
# 📌 공통 격자 및 시각화 함수

def get_density_map(df, lat_col, lon_col, grid_size=0.005):
    lat_min, lat_max = 33.15, 33.55
    lon_min, lon_max = 126.15, 126.95
    lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
    lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)
    hist, _, _ = np.histogram2d(
        df[lat_col], df[lon_col], bins=[lat_bins, lon_bins])
    return hist, lat_bins, lon_bins


def get_density_map_array(arr, grid_size=0.005):
    lat_min, lat_max = 33.15, 33.55
    lon_min, lon_max = 126.15, 126.95
    lat_bins = np.arange(lat_min, lat_max + grid_size, grid_size)
    lon_bins = np.arange(lon_min, lon_max + grid_size, grid_size)
    hist, _, _ = np.histogram2d(
        arr[:, 0], arr[:, 1], bins=[lat_bins, lon_bins])
    return hist, lat_bins, lon_bins


def make_skaterbox_points(matrix, lat_bins, lon_bins):
    points = []
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            val = matrix[i, j]
            if val > 0:
                lat = (lat_bins[i] + lat_bins[i+1]) / 2
                lon = (lon_bins[j] + lon_bins[j+1]) / 2
                points.append((lat, lon, val))
    return points

In [ ]:
# 격자 생성
demand_hist, lat_bins, lon_bins = get_density_map(df_demand, 'lat', 'lon')
supply_hist, _, _ = get_density_map_array(supply_points)

demand_points = make_skaterbox_points(demand_hist, lat_bins, lon_bins)
supply_points = make_skaterbox_points(supply_hist, lat_bins, lon_bins)

In [ ]:
# 지도1: 수요
m1 = folium.Map(location=[33.38, 126.55], zoom_start=11)
for lat, lon, val in demand_points:
    op = min(1.0, 0.2 + 0.8 * val / demand_hist.max())
    folium.CircleMarker([lat, lon], radius=3, color='blue',
                        fill=True, fill_opacity=op, weight=0).add_to(m1)
m1_path = "C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\01_skaterbox_demand_blue.html"
m1.save(m1_path)
print(f'{m1_path} saved.')

In [ ]:
# 지도2: 공급
m2 = folium.Map(location=[33.38, 126.55], zoom_start=11)
for lat, lon, val in supply_points:
    op = min(1.0, 0.2 + 0.8 * val / supply_hist.max())
    folium.CircleMarker([lat, lon], radius=3, color='red',
                        fill=True, fill_opacity=op, weight=0).add_to(m2)
m2_path = "C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\02_skaterbox_supply_red.html"
m2.save(m2_path)
print(f'{m2_path} saved.')

In [ ]:
# 지도3: 겹쳐서 시각화
m3 = folium.Map(location=[33.38, 126.55], zoom_start=11)
for lat, lon, val in demand_points:
    op = min(1.0, 0.2 + 0.8 * val / demand_hist.max())
    folium.CircleMarker([lat, lon], radius=3, color='blue',
                        fill=True, fill_opacity=op, weight=0).add_to(m3)
for lat, lon, val in supply_points:
    op = min(1.0, 0.2 + 0.8 * val / supply_hist.max())
    folium.CircleMarker([lat, lon], radius=3, color='red',
                        fill=True, fill_opacity=op, weight=0).add_to(m3)
m3_path = "C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\03_skaterbox_overlay_demand_supply.html"
m3.save(m3_path)
print(f'{m3_path} saved.')

In [ ]:
# # 지도4: Gap (공급 - 수요)
# gap_hist = supply_hist - demand_hist
# gap_points = make_skaterbox_points(gap_hist, lat_bins, lon_bins)

# m4 = folium.Map(location=[33.38, 126.55], zoom_start=11)
# colormap = cm.LinearColormap(colors=['red', 'white', 'blue'],
#                              vmin=-np.max(np.abs(gap_hist)),
#                              vmax=np.max(np.abs(gap_hist)),
#                              caption="공급 - 수요 Gap")
# colormap.add_to(m4)

# for lat, lon, val in gap_points:
#     folium.CircleMarker(
#         location=[lat, lon],
#         radius=4,
#         color=colormap(val),
#         fill=True,
#         fill_color=colormap(val),
#         fill_opacity=0.8,
#         popup=f"Gap: {val:.0f}"
#     ).add_to(m4)
# m4_path = "C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\04_skaterbox_gap_heatmap.html"
# m4.save(m4_path)

# m1_path, m2_path, m3_path, m4_path

('C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\01_skaterbox_demand_blue.html',
 'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\02_skaterbox_supply_red.html',
 'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\03_skaterbox_overlay_demand_supply.html',
 'C:\\Users\\charl\\OneDrive\\Desktop\\기타\\coding\\coding_on_study\\ziririgon\\First_project\\_5_19\\04_skaterbox_gap_heatmap.html')

In [ ]:
# Gap 시각화를 시각적으로 더 명확하게 만들기 위해 vmin/vmax 조정 또는 로그 스케일 적용
# 절댓값이 너무 작아 모두 흰색이 된 것에 대응하여 색상 범위 명확히 조정

# # 충전기 수 5배 부풀리기
# df_charger_count['충전기수'] *= 5  # ✅ 공급 강조

# 공급 좌표 생성
supply_points = np.repeat(
    df_charger_count[['lat', 'lon']].values,
    df_charger_count['충전기수'].astype(int),
    axis=0
)

# 1. GAP 계산 (공급 - 수요)
gap_hist = supply_hist - demand_hist

# 2. 모든 격자 좌표 + GAP 값 추출


def make_skaterbox_points_all(matrix, lat_bins, lon_bins):
    points = []
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            lat = (lat_bins[i] + lat_bins[i+1]) / 2
            lon = (lon_bins[j] + lon_bins[j+1]) / 2
            val = matrix[i, j]
            points.append((lat, lon, val))
    return points


gap_points = make_skaterbox_points_all(gap_hist, lat_bins, lon_bins)

# 3. 색상 기준 설정 (최소 대비 확보)
gap_max = np.max(np.abs(gap_hist))
vmax = max(gap_max, 10)
vmin = -vmax

# 4. 지도 생성 및 범례 추가
m_gap_colored = folium.Map(location=[33.38, 126.55], zoom_start=11)
colormap = cm.LinearColormap(['blue', 'white', 'red'],  # 🟦 수요 > 공급, ⚪ 같음, 🔴 공급 > 수요
                             vmin=vmin, vmax=vmax,
                             caption="수요 - 공급 Gap (파랑 = 수요 많음)")
colormap.add_to(m_gap_colored)

# 4. 점마다 수동 색상 지정
for lat, lon, val in gap_points:
    
    if val == 0:
        continue  # ✅ 아예 생략하거나 아래처럼 아주 연하게
        color = "white"
        opacity = 0.01  # 🎯 흰색은 아주 연하게
        
    elif val > 0:
        # 공급이 많다 → 빨강
        intensity = min(1.0, abs(val) / vmax)
        color = f"rgba(255, 0, 0, {intensity})"
        opacity = intensity
        
    else:
        # 수요가 많다 → 파랑
        intensity = min(1.0, abs(val) / vmax)
        color = f"rgba(0, 0, 255, {intensity})"
        opacity = intensity

    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=opacity,
        popup=f"Gap (수요-공급): {adjusted_gap:.0f}"
    ).add_to(m_gap_colored)

# 5. 저장
output_path = "C:/Users/charl/OneDrive/Desktop/기타/coding/coding_on_study/ziririgon/First_project/_5_19/04_skaterbox_gap_heatmap_better_fixed.html"
m_gap_colored.save(output_path)
print(f"✅ 저장 완료: {output_path}")

✅ 저장 완료: C:/Users/charl/OneDrive/Desktop/기타/coding/coding_on_study/ziririgon/First_project/_5_19/04_skaterbox_gap_heatmap_better_fixed.html


gap = 공급 히트맵 - 수요 히트맵
gap[i, j] = supply_hist[i, j] - demand_hist[i, j]
supply_hist: 해당 셀 내 충전소 개수 합 (충전기 수 기준으로 누적)
demand_hist: 해당 셀 내 수요 포인트 개수 합 (100m 간격으로 추출된 경로 포인트 수)

